In [4]:
# This is the script to populate driver's Data
# {driver_id, time, curr_lat, curr_long, dest, load}

import csv
import random
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
import time
import json
import random 
from elasticsearch import Elasticsearch

boundaries_file = "../kafka/boundaries.csv"
last_uid = 0 
kafka = KafkaClient('localhost:9092')
producer = KeyedProducer(kafka)
city = 'NYC'
total_drivers = 40
sleep = 2
step_to_dest = 2

es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', \
                    ' ip-172-31-0-105', 'ip-172-31-0-106'], \
                   port=9200)



def loadBoundaries(boundaries_file):
    boundaries = {}
    with open(boundaries_file, 'r') as f:
        reader = csv.reader(f, delimiter = ',')
        for row in reader:
            city = row[0]
            fence = row[1:] 
            boundaries.update({city:fence})
    f.close()
    return boundaries

def simulateTrip(id, city):
    step = 4
    bnd = bound[city]
    q = es.get(index='driver', doc_type='rolling', id=id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        d = q['_source']['destination']
        c = q['_source']['location']
        return [c[0] + ((c[0] - d[0])/step_to_dest), d[1] + ((d[1] - d[1])/step_to_dest)]
    else:
        return [random.uniform(float(bnd[0]), float(bnd[2])), \
                random.uniform(float(bnd[1]),float(bnd[3]))]
        
        
def generateDriver(city):
    global last_uid 
    last_uid += 1
    d_id = random.randint(1, total_drivers)
  
    
    driver_mapping ={ 
            'name': 'driver_{}'.format(d_id),
            'id': d_id,
            'status': 'idle',
            'location': simulateTrip(d_id, city),
            'ctime': str(datetime.now()),
            'p1': None,
            'p2': None,
            'destination': None,
            'destinationid': None,
            'altdest1': None,
            'altdest1id': None,
            'altdest2': None,
            'altdest2id': None,
        }

    q = es.get(index='driver', doc_type='rolling', id=d_id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        driver_mapping['status'] = q['_source']['status']
        driver_mapping['p1'] = q['_source']['p1']
        driver_mapping['p2'] = q['_source']['p2']
        driver_mapping['destination'] = q['_source']['destination']
    return(driver_mapping)



In [5]:
from kafka import KafkaConsumer
from elasticsearch import Elasticsearch
import os
import json
from datetime import datetime


#sc = SparkContext(appName="trip")
#ssc = StreamingContext(sc, 3)
es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', ' ip-172-31-0-105', 'ip-172-31-0-106'], port=9200)

class driver(object):
    def __init__(self, *arg, **kwargs):
        for item in arg:
            for key in item:
                setattr(self, key, item[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])
        try:
            res = datetime.strptime("{}".format(unicode(self.ctime)), '%Y-%m-%d %H:%M:%S.%f')
            self.ctime = res.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        except:
            pass
    def jsonFormat(self):
        return(json.dumps(self.__dict__))
    def isKnown(self):
        res = es.get(index='driver', doc_type='rolling', id=self.id, ignore=[404, 400])
        return(res['found'])
    def store(self):
        res = es.create(index='driver', doc_type='rolling', id=self.id, body=self.jsonFormat())
        return(res['created'])
    def update(self):
        q = '{{"doc": {}}}'.format(self.jsonFormat())
        res = es.update(index='driver', doc_type='rolling', id=self.id, body=q)
        return(res['_version'])
    def nearbyPassengers(self):
        geo_query = geoQuery(self.location, "5km")
        res = es.search(index='passenger', doc_type='rolling', body=geo_query )
        nearby = []
        for i in (res['hits']['hits']):
            nearby.append(i['_id'])
        return(nearby)
    
    def assignPassenger(self):
        if len(self.nearbyPassengers())>0: 
            p = getPassenger(self.nearbyPassengers()[0]) 
            p.status = 'pickup'
            self.status = 'pickup'
            p.driver = self.id
            self.destination = p.location
            self.destinationid = p.id
            p.update()
            self.update()
            return True
        else:
            return False

    def loadPassenger(self, p):
        if self.p1 == None: 
            self.p1 = p.id
            p.status = 'ontrip'
            self.status = 'ontrip'
            
        elif self.p2 == None:
            self.p2 = p.id
            p1 = getPassenger(self.p1)
            p1.match = p.id
            p.match = p1.id
            p1.status = 'match'
            p.status = 'match'
            self.status = 'full'
            p1.update()
        else:
            print('Cab is full')
            return False
        p.driver = self.id
        self.destination = p.destination
        self.destinationid = p.destinationid
        p.update()
        self.update()
        return True

    

class passenger(object):
    def __init__(self, *arg, **kwargs):
        for item in arg:
            for key in item:
                setattr(self, key, item[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])
        try:
            res = datetime.strptime("{}".format(unicode(self.ctime)), '%Y-%m-%d %H:%M:%S.%f')
            self.ctime = res.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        except:
            pass
    def jsonFormat(self):
        return(json.dumps(self.__dict__))
    def isKnown(self):
        res = es.get(index='passenger', doc_type='rolling', id=self.id, ignore=[404, 400])
        return(res['found'])
    def store(self):
        res = es.create(index='passenger', doc_type='rolling', id=self.id, body=self.jsonFormat())
        return(res['created'])
    def update(self):
        q = '{{"doc": {}}}'.format(self.jsonFormat())        
        res = es.update(index='passenger', doc_type='rolling', id=self.id, body=q)
        return(res['_version'])

def geoQuery(location, distance):
    geo_query = { "from" : 0, "size" : 3,
                 "_source":{"include": [ "_id" ]},
                 "query": {
            "filtered": {
                "query" : {
                    "term" : {"status": "wait"}
                },
                "filter": {
                    "geo_distance": {
                        "distance":      distance,
                        "distance_type": "plane", 
                        "location": location }
                           }
                         }
                           }
                }

    return(geo_query)

def getPassenger(p_id):
    res = es.get(index='passenger', doc_type='rolling', id=p_id, ignore=404)
    return(passenger(res['_source'])) if res['found'] else res['found']

def getDriver(p_id):
    res = es.get(index='driver', doc_type='rolling', id=p_id, ignore=404)
    return(driver(res['_source'])) if res['found'] else res['found']
    
def sanityCheck(driver):
    if driver.isKnown():
        driverRecord = getDriverRecord(driver.id)
        return(driver.time > driverRecord.time)
    else:
        return(True)    

def updateLocation(self):
    if isinstance(self, 'driver'):
        obj_ = getDriver(self.id)
    elif isinstance(self, 'passenger'):
        obj_ = getPassenger(self.id)    
    else:
        return(False)
    obj_.location = self.location
    obj_.update()
    return(True)

def arrived(d):
    p = getPassenger(d.p1)
    d.p1 == None
    d.p2 == None
    d.destination == None
    d.destinationid == None
    d.status == 'idle'
    p.status = 'arrived'
    self.update()
    p.update()
    if d.p2: 
        p2 = getPassenger(d.p2)
        p2.status == 'arrived'
        p2.update()
    return(True)


'''
    Driver:
    1. Check if timestamp make sense
    2. Check if driver exists in DB, if doesn't exists, create
    3. If cab's capacity is not zero, look for nearby requests
        1. If passenger request matched:
            1. Change status from idle to on trip (if necessary)
            2. Re-route / Set destination to passenger
            3. Save information to ElasticSearch
        2. If no nearby passenger:
            1. Update current location
    4. If cab's capacity is zero:
        Check if current location matches with destination:
        1. If matched, empty the cab and mark to idle
            1. Update number of trip
            2. Send trip info to kafka (for archive)
        2. If not matched, update current location
'''
def pipeDriver(x):
    d = driver(x)
    if d.isKnown():
        if d.status in ['idle']:
            d.assignPassenger()
        elif d.status in ['pickup']:
            if not d.location == d.destination:
                d.update()
            else:
                p = getPassenger(d.p2) if d.p2 else getPassenger(d.p1)
                d.loadPassenger(d, p)

        elif d.status in ['ontrip']:
            if d.location == d.destination:
                arrived(d)
            elif not d.p2: 
                p = getPassenger(d.p1)
                p.location = d.location
                self.update()
                p.update()
                d.assignPassenger()
            else:
                p = getPassenger(d.p1)
                p.location = d.location
                p2 = getPassenger(d.p2)
                p2.location = d.location
                p.update()
                p2.update()
                self.update()
    else:
        d.store()

def pipePassenger(x):
    p = passenger(x)
    if not p.isKnown():
        p.store()
    

In [6]:
bound = loadBoundaries(boundaries_file)
for n in range(total_drivers):
    driver = generateDriver(city)
    u_json = json.dumps(driver).encode('utf-8')
    key = json.dumps(city).encode('utf-8')
    print('{}'.format(driver))
    producer.send(b'driver', key, u_json) 
    #time.sleep(sleep)


{'status': u'pickup', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 39, 'name': 'driver_39', 'p2': None, 'p1': None, 'ctime': '2016-09-24 05:46:10.751360', 'altdest1id': None, 'destination': [40.714291194869844, -73.97192672679024], 'destinationid': None, 'location': [40.851388019652205, -73.97192672679024]}
{'status': 'idle', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 34, 'name': 'driver_34', 'p2': None, 'p1': None, 'ctime': '2016-09-24 05:46:11.017946', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.722662614118136, -73.97671755328074]}
{'status': u'pickup', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 4, 'name': 'driver_4', 'p2': None, 'p1': None, 'ctime': '2016-09-24 05:46:11.058039', 'altdest1id': None, 'destination': [40.76332246638906, -73.99906432472923], 'destinationid': None, 'location': [40.81415838819028, -73.99906432472923]}
{'status': 'idle', 'altdest2id': None, 'altdest2': None, 'altdest

In [ ]:

    consumer = KafkaConsumer('driver', group_id = 1)
    for message in consumer:
        d = json.loads(message.value)
        #print "{}".format(d)
        y = pipeDriver(d)

        consumer.commit()
    consumer.close()

